In [2]:
import torch
import torch.nn as nn
# !pip install easyfsl
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import Omniglot
from torchvision.models import resnet18
from tqdm import tqdm

from easyfsl.samplers import TaskSampler
from easyfsl.utils import plot_images, sliding_average

class PrototypicalNetworks_dynamic_query(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks_dynamic_query, self).__init__()
        self.backbone = backbone

    def forward(
        self,
        support_images,
        support_labels,
        query_images: torch.Tensor,
    ) -> torch.Tensor:
        """
        Predict query labels using labeled support images.
        """
        num_classes=len(list(support_labels))
        prototypes = []
        for class_label in range(num_classes):
            # print(type(support_images[class_label]),torch.cat(support_images[class_label].shape)
            support_images_eachclass=(support_images[class_label])
            # print("kk", support_images_eachclass.shape)
            class_features = self.backbone(support_images_eachclass.to(device))  # Select features for the current class
            class_prototype = class_features.mean(dim=0)  # Compute the mean along the batch dimension
            prototypes.append(class_prototype)
        prototypes = torch.stack(prototypes)


        distances=[]
        for each_query_class in query_images:

          query_features = self.backbone(each_query_class)

          # Compute the distance between query features and prototypes
          distance = torch.cdist(query_features, prototypes)
          distances.append(-distance)

        # print(distances.shape)

        return distances


In [1]:
root_path = '/home/asufian/Desktop/output_olchiki/olchiki'

In [3]:
import os
import random
from collections import defaultdict
from PIL import Image, ImageOps
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class PrototypicalOmniglotDataset(Dataset):
    def __init__(self, root, num_classes=1623, n_shot=5, n_query=10, transform=None):
        self.root = root
        self.num_classes = num_classes
        self.n_shot = n_shot
        self.n_query = n_query
        self.transform = transform
        self.samples_by_label = defaultdict(list)
        self.all_imgs = {}
        self.classes = []

        # Common image file extensions
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff', '.webp']

        # Organize data into classes and image paths
        for alphabet in os.listdir(self.root):
            alphabet_path = os.path.join(self.root, alphabet)
            if not os.path.isdir(alphabet_path):
                continue

            for char_class in os.listdir(alphabet_path):
                char_class_path = os.path.join(alphabet_path, char_class)

                # Skip if the entry is not a directory
                if not os.path.isdir(char_class_path):
                    continue

                all_images = []
                for img_name in sorted(os.listdir(char_class_path)):
                    img_path = os.path.join(char_class_path, img_name)

                    # Check if it's a file with a recognized image extension
                    if os.path.isfile(img_path) and any(img_name.lower().endswith(ext) for ext in image_extensions):
                        all_images.append(img_path)

                if all_images:
                    char_class_name = f"{alphabet}_{char_class}"
                    self.samples_by_label[char_class_name] = list(range(len(all_images)))
                    self.all_imgs[char_class_name] = all_images
                    self.classes.append(char_class_name)
                    # print(char_class_name)

    def transform_image(self, raw_img):
        img = raw_img#ImageOps.invert(raw_img)
        if self.transform is not None:
            # print(self.transform)
            img = self.transform(img)
        # print(img.shape)
        return img

    def __getitem__(self, selected_classes,selected_supports):#
        # selected_classes = random.sample(self.classes, self.num_classes)
        class_indices = [self.samples_by_label[each_cls] for each_cls in selected_classes]
        # print("selected_classes: ",selected_classes)
        support_set = []
        query_set = []
        label_id = 0
        qs=[]
        ko=0
        ghj=[]
        for idx_set in class_indices:
            # Creating support set
            selected_support =selected_supports[ko]# random.sample(idx_set, self.n_shot)
            ko=ko+1
            ghj.append(selected_support)
            # print("selected_support images: ", ghj)
            support_images = [self.transform_image(Image.open(self.all_imgs[selected_classes[label_id]][each]).convert('L')) for each in selected_support]

            # Creating query set
            selected_query = [item for item in idx_set if item not in selected_support]
            query_images = [self.transform_image(Image.open(self.all_imgs[selected_classes[label_id]][each]).convert('L')) for each in selected_query]
            support_set.append((support_images, [label_id for _ in range(self.n_shot)]))
            # print("aaaaaaaaaa",label_id)
            query_set.append((query_images,  [label_id for _ in range(len(selected_query))]))

            label_id += 1
            # qs= [item for item in a for _ in range(self.n_query)]
        return support_set, query_set

    def __len__(self):
        return 10

# Set up the transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor(),
])

# Set up the SiameseOmniglotDataset
# Training == evaluation
# root_path = './assamese'
num_classes = 5  # Set the desired number of classes per episode
n_shot = 5
n_query = 10

PrototypicalOmniglotDatasetLoader = PrototypicalOmniglotDataset(root=root_path, num_classes=num_classes, n_shot=n_shot, n_query=n_query, transform=transform)


In [4]:
import torch
import torch.nn as nn

class PrototypicalNetworks_dynamic_query(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks_dynamic_query, self).__init__()
        self.backbone = backbone

    def forward(
        self,
        support_images,
        support_labels,
        query_images: torch.Tensor,
    ) -> torch.Tensor:
        """
        Predict query labels using labeled support images.
        """
        num_classes=len(list(support_labels))
        prototypes = []
        for class_label in range(num_classes):
            # print(type(support_images[class_label]),torch.cat(support_images[class_label].shape)
            support_images_eachclass=(support_images[class_label])
            # print("kk", support_images_eachclass.shape)
            class_features = self.backbone(support_images_eachclass.to(device))  # Select features for the current class
            class_prototype = class_features.mean(dim=0)  # Compute the mean along the batch dimension
            prototypes.append(class_prototype)
        prototypes = torch.stack(prototypes)


        distances=[]
        for each_query_class in query_images:

          query_features = self.backbone(each_query_class)

          # Compute the distance between query features and prototypes
          distance = torch.cdist(query_features, prototypes)
          distances.append(-distance)

        # print(distances.shape)

        return distances


In [5]:
class PrototypicalNetworks33(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks33, self).__init__()
        self.backbone = backbone

    def forward(
        self,
        support_images,
        support_labels,
        query_images: torch.Tensor,
    ) -> torch.Tensor:
        num_classes = len(support_labels)
        prototypes = []
        for class_label in range(num_classes):
            support_images_eachclass = support_images[class_label]
            class_features = torch.stack([self.backbone(each.unsqueeze(0)) for each in support_images_eachclass])
            prototype_bymean = class_features.mean(dim=0)
            # print(prototype_bymean.shape)
            prototypes.append(prototype_bymean)

        query_features = torch.stack([self.backbone(each.unsqueeze(0)) for each in query_images])
        # print(query_features.shape,query_features[0].shape,torch.stack(prototypes).shape,torch.stack(prototypes)[0].shape)
        # distances = -torch.cdist(query_features, torch.stack(prototypes))
        distances = []  # Initialize list to store distances
        prototypes=torch.stack(prototypes)
        for each_query_feature in query_features:
            d = torch.cdist(prototypes, each_query_feature)  # Compute distances
            distances.append(d.squeeze())  # Remove singleton dimension and store
            # print(d.shape)  # Output: [5, 1], shape of each distance matrix

        distances = torch.stack(distances)

        return -distances


In [45]:
import pandas as pd
import os
import csv
def save_in_csv(cls,flnm,conf_matrix, acciuracy,precision_overall, recall_overall, f_betas):
  with open(flnm, 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      headers=['Accuracy', 'Precision', 'Recall']
      beta=-5
      values=[]
      values.append(acciuracy)
      values.append(precision_overall)
      values.append(recall_overall)
      for each in f_betas:
              headers.append('F-betas (' +str(beta)+' )')
              beta=beta+1
              values.append(each)

    # Write confusion matrix
      writer.writerow(['used Class:'])
      writer.writerows([cls])
      writer.writerow([])
      writer.writerow([])
      writer.writerow(['Confusion Matrix'])
      writer.writerows(conf_matrix)
      writer.writerow([])
      writer.writerow([])  # Add an empty row for separation

      # Write metrics

      writer.writerow(headers)
      writer.writerow(values)


  # print("Metrics have been written to 'metrics.csv' file.")
def save_in_text_file(msg,cls, flnm, conf_matrix, accuracy, precision_overall, recall_overall, f_betas):
    with open(flnm, 'a') as textfile:
        textfile.write(str("\n\n\n                                 "+msg))
        textfile.write("\n\n\n")
        textfile.write("used Class:\n")
        textfile.write(str(cls) + "\n\n")

        textfile.write("Confusion Matrix:\n")
        for row in conf_matrix:
            textfile.write(' '.join([str(elem) for elem in row]) + "\n")
        textfile.write("\n")

        textfile.write("Metrics:\n")
        textfile.write("Accuracy: {}\n".format(accuracy))
        textfile.write("Precision: {}\n".format(precision_overall))
        textfile.write("Recall: {}\n".format(recall_overall))

        beta = -5
        for f_beta in f_betas:
            textfile.write("F-betas ({}): {}\n".format(beta, f_beta))
            beta += 1

    # print("Metrics have been written to '{}' file.".format(flnm))

def save_in_excel_final(msg, cls, flnm, conf_matrix, accuracy, precision_overall, recall_overall):
    data = {
        "Used Class": [cls],
        "Used support images idx": [msg],
        "Confusion Matrix": [conf_matrix],
        "Accuracy": [accuracy],
        "Precision": [precision_overall],
        "Recall": [recall_overall]
    }
    # for i, f_beta in enumerate(f_betas):
    #     data[f"F-beta ({i - 5})"] = [f_beta]

    df_new = pd.DataFrame(data)
    df_new = df_new.transpose()  # Transpose the DataFrame

    if os.path.exists(flnm):
        df_existing = pd.read_excel(flnm, index_col=0)  # Read existing file
        df_combined = pd.concat([df_existing, df_new], axis=1)  # Concatenate along columns
        df_combined.to_excel(flnm)  # Write to Excel
    else:
        df_new.to_excel(flnm)  # Write new DataFrame to Excel

    # print("Metrics have been written to '{}' file.".format(flnm))


In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch
import torch.nn as nn
import torch.hub

# Define a modified version of DenseNet169 with dropout after each convolutional layer
class DenseNet169WithDropout(nn.Module):
    def __init__(self,modell, pretrained=False, dr=0.4):
        super(DenseNet169WithDropout, self).__init__()
        self.densenet169 = torch.hub.load('pytorch/vision:v0.10.0', modell, pretrained=pretrained)
        self.dropout = nn.Dropout(p=dr)  # Dropout with probability 0.5

        # Modify each dense block to include dropout after each convolutional layer
        for name, module in self.densenet169.features.named_children():
            if isinstance(module, nn.Sequential):
                for sub_name, sub_module in module.named_children():
                    if isinstance(sub_module, nn.Conv2d):
                        setattr(module, sub_name, nn.Sequential(sub_module, self.dropout))

    def forward(self, x):
        return self.densenet169(x)

# Instantiate the modified DenseNet169 model with dropout
# convolutional_network_with_dropout = DenseNet169WithDropout(modell='resnet18',pretrained=False,dr=0.4)

# Example usage:
# output = convolutional_network_with_dropout(input_tensor)

class ResNet18WithDropout(nn.Module):
    def __init__(self, pretrained=False, dr=0.4):
        super(ResNet18WithDropout, self).__init__()
        self.resnet18 = models.resnet18(pretrained=pretrained)
        self.dropout = nn.Dropout(p=dr)

        # Modify each residual block to include dropout after each convolutional layer
        for name, module in self.resnet18.named_children():
            if isinstance(module, nn.Sequential):
                for sub_name, sub_module in module.named_children():
                    if isinstance(sub_module, nn.Conv2d):
                        setattr(module, sub_name, nn.Sequential(sub_module, self.dropout))

    def forward(self, x):
        return self.resnet18(x)

# Instantiate the modified ResNet18 model with dropout
# convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.4)


In [102]:
clss_5=[
  ['Olchiki_2_80_c28', 'Olchiki_2_80_c1', 'Olchiki_2_80_c25', 'Olchiki_2_80_c17', 'Olchiki_2_80_c21'],
    ['Olchiki_2_80_c17', 'Olchiki_2_80_c5', 'Olchiki_2_80_c20', 'Olchiki_2_80_c16', 'Olchiki_2_80_c6'],
    ['Olchiki_2_80_c7', 'Olchiki_2_80_c30', 'Olchiki_2_80_c17', 'Olchiki_2_80_c15', 'Olchiki_2_80_c21'],
    ['Olchiki_2_80_c11', 'Olchiki_2_80_c26', 'Olchiki_2_80_c7', 'Olchiki_2_80_c25', 'Olchiki_2_80_c4'],
    ['Olchiki_2_80_c26', 'Olchiki_2_80_c9', 'Olchiki_2_80_c18', 'Olchiki_2_80_c1', 'Olchiki_2_80_c22'],
    ['Olchiki_2_80_c30', 'Olchiki_2_80_c29', 'Olchiki_2_80_c11', 'Olchiki_2_80_c6', 'Olchiki_2_80_c19'],
    ['Olchiki_2_80_c26', 'Olchiki_2_80_c3', 'Olchiki_2_80_c15', 'Olchiki_2_80_c5', 'Olchiki_2_80_c7'],
    ['Olchiki_2_80_c10', 'Olchiki_2_80_c26', 'Olchiki_2_80_c28', 'Olchiki_2_80_c29', 'Olchiki_2_80_c25'],
    ['Olchiki_2_80_c18', 'Olchiki_2_80_c11', 'Olchiki_2_80_c1', 'Olchiki_2_80_c24', 'Olchiki_2_80_c29'],
    ['Olchiki_2_80_c13', 'Olchiki_2_80_c14', 'Olchiki_2_80_c16', 'Olchiki_2_80_c24', 'Olchiki_2_80_c12'],
     ['Olchiki_2_80_c16', 'Olchiki_2_80_c22', 'Olchiki_2_80_c20', 'Olchiki_2_80_c26', 'Olchiki_2_80_c12'],
    ['Olchiki_2_80_c3', 'Olchiki_2_80_c22', 'Olchiki_2_80_c18', 'Olchiki_2_80_c26', 'Olchiki_2_80_c19']

     ]

clss_support_imagesss_1_shot=[
 [[24], [37], [78], [65], [37]],
    [[15], [4], [39], [60], [23]],
    [[1], [48], [29], [76], [27]],
    [[71], [58], [35], [24], [3]],
    [[23], [40], [58], [35], [56]],
    [[66], [63], [23], [72], [64]],
    [[21], [45], [37], [8], [63]],
     [[9], [19], [67], [51], [20]],
    [[55], [34], [7], [10], [9]],
    [[47], [25], [48], [10], [54]],
    [[13], [33], [72], [51], [20]],
    [[59], [5], [52], [70], [1]]







]

clss_support_imagesss_5_shot=[
    [[24, 17, 73, 60, 18], [37, 38, 24, 4, 19], [78, 45, 50, 6, 14], [65, 62, 41, 33, 55], [37, 27, 61, 65, 10]],
    [[15, 30, 21, 54, 33], [4, 62, 38, 57, 34], [39, 8, 75, 37, 10], [60, 67, 14, 10, 53], [23, 42, 44, 1, 5]],
    [[1, 63, 32, 67, 5], [48, 63, 25, 13, 22], [29, 17, 12, 30, 75], [76, 49, 24, 18, 30], [27, 56, 4, 41, 66]],
    [[71, 78, 55, 22, 61], [58, 70, 0, 54, 17], [35, 31, 26, 16, 30], [24, 35, 49, 32, 30], [3, 50, 39, 18, 22]],
    [[23, 2, 59, 68, 74], [40, 14, 4, 3, 45], [58, 51, 31, 45, 75], [35, 46, 61, 55, 79], [56, 15, 64, 33, 14]],
    [[66, 61, 29, 8, 39], [63, 77, 69, 36, 72], [23, 56, 28, 52, 31], [72, 20, 41, 73, 49], [64, 48, 53, 13, 8]],
    [[21, 17, 74, 56, 30], [45, 37, 63, 30, 8], [37, 42, 5, 52, 50], [8, 46, 52, 36, 20], [63, 26, 43, 8, 17]],
    [[9, 47, 25, 35, 46], [19, 27, 35, 34, 40], [67, 4, 16, 61, 60], [51, 53, 4, 32, 20], [20, 40, 25, 29, 39]],
    [[55, 44, 15, 8, 62], [34, 9, 3, 16, 46], [7, 76, 52, 3, 16], [10, 11, 34, 37, 77], [9, 41, 67, 6, 36]],
     [[47, 49, 72, 36, 9], [25, 41, 71, 40, 57], [48, 24, 27, 41, 52], [10, 34, 11, 59, 7], [54, 55, 11, 64, 16]],
    [[13, 35, 18, 11, 42], [33, 50, 28, 27, 53], [72, 58, 64, 33, 73], [51, 13, 31, 66, 58], [20, 54, 45, 27, 0]],
     [[59, 73, 74, 0, 47], [5, 20, 27, 3, 71], [52, 0, 2, 38, 60], [70, 1, 30, 64, 6], [1, 34, 8, 56, 49]],
]

clss_support_imagesss_10_shot=[
    [[24, 17, 73, 60, 18, 68, 23, 39, 41, 20], [37, 38, 24, 4, 19, 3, 17, 35, 66, 56], [78, 45, 50, 6, 14, 54, 32, 25, 13, 55], [65, 62, 41, 33, 55, 25, 51, 74, 48, 35], [37, 27, 61, 65, 10, 63, 70, 24, 57, 8]],
    [[15, 30, 21, 54, 33, 28, 48, 36, 9, 37], [4, 62, 38, 57, 34, 59, 32, 9, 3, 68], [39, 8, 75, 37, 10, 27, 79, 26, 13, 6], [60, 67, 14, 10, 53, 71, 33, 42, 1, 0], [23, 42, 44, 1, 5, 32, 46, 36, 13, 34]],
    [[1, 63, 32, 67, 5, 74, 43, 51, 68, 8], [48, 63, 25, 13, 22, 4, 1, 68, 62, 38], [29, 17, 12, 30, 75, 31, 67, 11, 68, 37], [76, 49, 24, 18, 30, 6, 38, 26, 15, 7], [27, 56, 4, 41, 66, 28, 73, 50, 29, 26]],
     [[71, 78, 55, 22, 61, 69, 79, 33, 24, 13], [58, 70, 0, 54, 17, 62, 47, 7, 60, 78], [35, 31, 26, 16, 30, 2, 44, 14, 10, 12], [24, 35, 49, 32, 30, 48, 14, 72, 29, 68], [3, 50, 39, 18, 22, 63, 25, 77, 47, 26]],
     [[23, 2, 59, 68, 74, 36, 41, 45, 72, 27], [40, 14, 4, 3, 45, 53, 21, 77, 13, 12], [58, 51, 31, 45, 75, 9, 44, 40, 61, 19], [35, 46, 61, 55, 79, 12, 76, 0, 47, 33], [56, 15, 64, 33, 14, 2, 26, 25, 8, 20]],
    [[66, 61, 29, 8, 39, 53, 59, 10, 25, 24], [63, 77, 69, 36, 72, 15, 60, 58, 16, 49], [23, 56, 28, 52, 31, 2, 6, 21, 7, 68], [72, 20, 41, 73, 49, 35, 21, 34, 29, 50], [64, 48, 53, 13, 8, 54, 11, 9, 29, 65]],
    [[21, 17, 74, 56, 30, 23, 64, 38, 67, 1], [45, 37, 63, 30, 8, 12, 70, 4, 1, 75], [37, 42, 5, 52, 50, 76, 43, 46, 22, 61], [8, 46, 52, 36, 20, 5, 1, 31, 45, 67], [63, 26, 43, 8, 17, 22, 16, 54, 70, 32]],
     [[9, 47, 25, 35, 46, 7, 2, 27, 49, 69], [19, 27, 35, 34, 40, 53, 16, 7, 64, 45], [67, 4, 16, 61, 60, 79, 12, 70, 63, 42], [51, 53, 4, 32, 20, 1, 68, 15, 55, 10], [20, 40, 25, 29, 39, 43, 3, 23, 38, 0]],
    [[55, 44, 15, 8, 62, 77, 58, 71, 69, 30], [34, 9, 3, 16, 46, 2, 8, 76, 23, 32], [7, 76, 52, 3, 16, 28, 19, 18, 41, 26], [10, 11, 34, 37, 77, 48, 9, 23, 4, 71], [9, 41, 67, 6, 36, 56, 49, 10, 53, 45]],
    [[47, 49, 72, 36, 9, 61, 56, 40, 30, 68], [25, 41, 71, 40, 57, 44, 37, 45, 56, 3], [48, 24, 27, 41, 52, 58, 32, 74, 9, 12], [10, 34, 11, 59, 7, 30, 24, 68, 69, 20], [54, 55, 11, 64, 16, 51, 33, 22, 39, 35]],
     [[13, 35, 18, 11, 42, 77, 59, 66, 71, 74], [33, 50, 28, 27, 53, 70, 0, 22, 2, 52], [72, 58, 64, 33, 73, 54, 78, 53, 49, 22], [51, 13, 31, 66, 58, 4, 18, 44, 32, 52], [20, 54, 45, 27, 0, 39, 77, 9, 67, 29]],
    [[59, 73, 74, 0, 47, 42, 39, 19, 48, 75], [5, 20, 27, 3, 71, 10, 19, 22, 11, 25], [52, 0, 2, 38, 60, 6, 25, 37, 46, 18], [70, 1, 30, 64, 6, 29, 50, 8, 34, 38], [1, 34, 8, 56, 49, 53, 36, 5, 15, 32]],

]



In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelr_res18.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/olchiki/ressecondary/model_1/model_B_olchiki_1-shot_res.pth',map_location=torch.device('cpu')))

torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

In [25]:
# !ls '/home/asufian/Desktop/output_olchiki/'

In [50]:
criterion = nn.CrossEntropyLoss()

In [51]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np
import numpy as np
import pandas as pd
# root_path = '/content/assamese'
num_classes = 5
n_shot = 1
n_query = 14
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transformt_ugh = transforms.Compose([
    transforms.Resize((64, 64)),
    # transforms.RandomResizedCrop(64),  # Randomly crop the image and resize to 224x224
    # transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    # transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),  # Randomly adjust brightness, contrast, saturation, and hue
    # transforms.RandomAffine(degrees=0, translate=(0, 0), scale=(0.9, 1.1), shear=0),  # Random zoom (scaling)
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the image with ImageNet statistics
])

print(f"Generated random values - num_classes: {num_classes}, n_shot: {n_shot}, n_query: {n_query}")

test_loader = PrototypicalOmniglotDataset(root=root_path, num_classes=num_classes, n_shot=n_shot, n_query=20, transform=transformt_ugh)

def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    precision=0
    recall=0
    
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

            # msg="\n\n       Way: "+str(len(clss_5[tlv_cls]))+"  Shot: "+str(len(clss_support_imagesss_1_shot[tlv_cls][0]))    +" \n\n"
            # msg=str(clss_5[tlv_cls])
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision=precision+precision_overall
            recall=recall+recall_overall
    print("---------------------->",ttlac/ttla,precision/ttla ,recall/ttla)
# %%%%%%%%%%%%%%%%%%%


pp='/home/asufian/Desktop/output_olchiki/faltu'






Generated random values - num_classes: 5, n_shot: 1, n_query: 14


In [52]:
import numpy as np

conf_matrix = np.array([[100, 0, 0, 0, 0],
                        [0, 100, 0, 0, 0],
                        [0, 0, 100, 0, 0],
                        [0, 0, 0, 100, 0],
                        [0, 0, 0, 0, 100]])

def calculate_metrics_get_per(msg, conf_matrix,acciuracy, cls, flnm):
    if conf_matrix.size == 0:
        raise ValueError("Confusion matrix is empty!")

    metrics = []
    for i in range(conf_matrix.shape[0]):
        tp = conf_matrix[i, i]
        fp = np.sum(conf_matrix[:, i]) - tp
        fn = np.sum(conf_matrix[i, :]) - tp

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        metrics.append((precision, recall, f1))

    # Compute overall scores
    precision_overall = np.mean([m[0] for m in metrics])
    recall_overall = np.mean([m[1] for m in metrics])
    f1_overall = np.mean([m[2] for m in metrics])

    # Compute overall accuracy
    total_samples = np.sum(conf_matrix)
    correct_predictions = np.sum(np.diag(conf_matrix))
    accuracy_overall = correct_predictions / total_samples if total_samples > 0 else 0
    # print(accuracy_overall)

    # Save results (Assuming these functions exist)
    save_in_excel_final(msg, cls, flnm + ".xlsx", conf_matrix, accuracy_overall, precision_overall, recall_overall)
    # save_in_text_file(msg, cls, flnm + ".txt", conf_matrix, accuracy_overall, precision_overall, recall_overall, f_betas)

    return precision_overall, recall_overall, f1_overall


# precision, recall, f_beta = calculate_metrics(conf_matrix)
# print(f'Overall Metrics for beta={beta}:')
# print(f'Overall Precision: {precision:.4f}')
# print(f'Overall Recall: {recall:.4f}')
# print(f'Overall F-beta ({f_beta}): ')
# print()


In [53]:


def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

           
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

            
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [54]:
def update_model_weights2(model_a, model_b, conv_ratio=0.8, fc_ratio=0.8, bias_ratio=0.8):
    """Updates model_a's weights using a weighted combination of model_a and model_b."""
    for name, param in model_a.named_parameters():
        if name in model_b.state_dict():
            weight_b = model_b.state_dict()[name]
            
            if 'weight' in name:
                # Separate conditions for convolutional and fully connected layers
                if 'conv' in name:  # Convolutional layer weights
                    # print('conv')
                    param.data = conv_ratio * weight_b + (1 - conv_ratio) * param.data
                elif 'fc' in name or 'linear' in name:  # Fully connected layer weights
                    # print('fc')
                    param.data = fc_ratio * weight_b + (1 - fc_ratio) * param.data
            
            elif 'bias' in name:
                param.data = bias_ratio * weight_b + (1 - bias_ratio) * param.data

In [55]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to get model accuracy
def get_model_accuracy(b_a_r, c_a_r, f_a_r):
    try:
        print(f"Testing b_a_r = {b_a_r:.4f}, c_a_r = {c_a_r:.4f}, f_a_r = {f_a_r:.4f}")  # Debugging Output

        convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
        convolutional_network_with_dropout.fc = nn.Flatten()
        xyz = convolutional_network_with_dropout
        M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
        M3.load_state_dict(torch.load('model_own_path.pt'))

        # Load Model M2 (Reference)
        convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
        convolutional_network_with_dropout.fc = nn.Flatten()
        xyz2 = convolutional_network_with_dropout
        M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
        M2.load_state_dict(torch.load('model_mu_path.pt'))

        # Apply bias adaptation to M3 using M2
        update_model_weights2(M3, M2, conv_ratio=c_a_r, fc_ratio=f_a_r, bias_ratio=b_a_r)
        
        # Evaluate accuracy
        accuracy = evaluate3(pp+'/_olchiki_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)
        
        if accuracy is None:
            raise ValueError("Accuracy computation failed. Returning worst case.")

        return accuracy  # Ensure accuracy is valid
    except Exception as e:
        print(f"Error encountered: {e}. Assigning worst-case accuracy of 0.")
        return 0.0  # Assign lowest accuracy to prevent optimization failures


In [31]:

# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.709282700421941
Evaluated Accuracy: 0.7093
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000
----------------------> 0.6369198312236287
Evaluated Accuracy: 0.6369
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429
----------------------> 0.6913502109704642
Evaluated Accuracy: 0.6914
Testing b_a_r = 0.6509, c_a_r = 0.0564, f_a_r = 0.7220
----------------------> 0.7493670886075949
Evaluated Accuracy: 0.7494
Testing b_a_r = 0.9386, c_a_r = 0.0008, f_a_r = 0.9922
----------------------> 0.7261603375527427
Evaluated Accuracy: 0.7262
Testing b_a_r = 0.2729, c_a_r = 0.0992, f_a_r = 0.0805


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.7147679324894515
Evaluated Accuracy: 0.7148
Testing b_a_r = 0.2213, c_a_r = 1.0000, f_a_r = 0.9428


...

----------------------> 0.7506329113924051
Evaluated Accuracy: 0.7506
Testing b_a_r = 0.0916, c_a_r = 0.0444, f_a_r = 0.0036


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.6991561181434599
Evaluated Accuracy: 0.6992
Testing b_a_r = 0.1036, c_a_r = 0.9006, f_a_r = 0.9947


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.4198312236286921
Evaluated Accuracy: 0.4198

✅ Optimal Values:
   - b_a_r: 0.9169
   - c_a_r: 0.1354
   - f_a_r: 0.5615
📈 Highest Accuracy Achieved: 0.7627


In [56]:
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_olchiki/resnet_resluts/base/5_w_1_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/secondary/5_w_1_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/amul/5_w_1_s_f", test_loader, M3, criterion)


# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_olchiki_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

---Accu-----pre----rec---------> 0.6846 ± 0.0834  0.7269 ± 0.0951  0.6846 ± 0.0834


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.4696 ± 0.0886  0.5225 ± 0.0912  0.4696 ± 0.0886
---Accu-----pre----rec---------> 0.7627 ± 0.0978  0.8044 ± 0.0788  0.7627 ± 0.0978


In [35]:
!mkdir '/home/asufian/Desktop/output_olchiki/resnet_resluts/base'

In [21]:
!ls /home/asufian/Desktop/output_olchiki/code/olchiki/model_5/model_B_olchiki_1-shot_30.pth

model_B_olchiki_1-shot_20.pth  model_B_olchiki_1-shot_80.pth
model_B_olchiki_1-shot_30.pth  model_B_olchiki_1-shot_90.pth
model_B_olchiki_1-shot_50.pth


In [ ]:
########################  5   way    5  shot    #########################

In [103]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelr_res18.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/olchiki/ressecondary/model_5/model_B_olchiki_5-shot_res.pth',map_location=torch.device('cpu')))

torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [104]:


def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

           
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

            
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [105]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.8915555555555555
Evaluated Accuracy: 0.8916
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000
----------------------> 0.6746666666666666
Evaluated Accuracy: 0.6747
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429
----------------------> 0.7553333333333333
Evaluated Accuracy: 0.7553
Testing b_a_r = 0.6509, c_a_r = 0.0564, f_a_r = 0.7220
----------------------> 0.8220000000000001
Evaluated Accuracy: 0.8220
Testing b_a_r = 0.9386, c_a_r = 0.0008, f_a_r = 0.9922
----------------------> 0.7737777777777778
Evaluated Accuracy: 0.7738
Testing b_a_r = 0.2849, c_a_r = 0.9362, f_a_r = 0.8249


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.39888888888888885
Evaluated Accuracy: 0.3989
Testing b_a_r = 0.0000, c_a_r = 0.1520, f_a_r = 0.0000


...

----------------------> 0.44511111111111107
Evaluated Accuracy: 0.4451
Testing b_a_r = 0.0069, c_a_r = 0.1618, f_a_r = 0.7130


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.9017777777777777
Evaluated Accuracy: 0.9018
Testing b_a_r = 0.0415, c_a_r = 0.1441, f_a_r = 0.8108


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.8984444444444445
Evaluated Accuracy: 0.8984

✅ Optimal Values:
   - b_a_r: 0.0000
   - c_a_r: 0.1521
   - f_a_r: 0.7508
📈 Highest Accuracy Achieved: 0.9024


In [ ]:
# b_a_r: 
#    - c_a_r: 
#    - f_a_r: 

#  pt=20

In [106]:

convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_olchiki/resnet_resluts/base/5_w_5_s_f_ff', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/secondary/5_w_5_s_f_ff", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/amul/5_w_5_s_f_ff", test_loader, M3, criterion)




# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_olchiki_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.7591 ± 0.0982  0.7797 ± 0.0954  0.7591 ± 0.0982


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.6169 ± 0.1020  0.6402 ± 0.1033  0.6169 ± 0.1020
---Accu-----pre----rec---------> 0.9024 ± 0.0558  0.9085 ± 0.0513  0.9024 ± 0.0558


In [33]:
!ls /home/asufian/Desktop/output_olchiki/code/olchiki/model_10

model_B_olchiki_1-shot_20.pth  model_B_olchiki_1-shot_70.pth
model_B_olchiki_1-shot_40.pth  model_B_olchiki_1-shot_90.pth
model_B_olchiki_1-shot_50.pth


In [71]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelr_res18.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/olchiki/ressecondary/model_10/model_B_olchiki_10-shot_res.pth',map_location=torch.device('cpu')))


torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

In [72]:


def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

           
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

            
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [73]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.9226190476190478
Evaluated Accuracy: 0.9226
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000
----------------------> 0.6683333333333334
Evaluated Accuracy: 0.6683
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429
----------------------> 0.7997619047619047
Evaluated Accuracy: 0.7998
Testing b_a_r = 0.6509, c_a_r = 0.0564, f_a_r = 0.7220
----------------------> 0.9140476190476191
Evaluated Accuracy: 0.9140
Testing b_a_r = 0.9386, c_a_r = 0.0008, f_a_r = 0.9922
----------------------> 0.8707142857142857
Evaluated Accuracy: 0.8707
Testing b_a_r = 0.0250, c_a_r = 1.0000, f_a_r = 0.4630


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.4033333333333334
Evaluated Accuracy: 0.4033
Testing b_a_r = 0.0000, c_a_r = 0.1267, f_a_r = 0.0000


...

----------------------> 0.9330952380952383
Evaluated Accuracy: 0.9331
Testing b_a_r = 0.9910, c_a_r = 0.0809, f_a_r = 0.9840


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.9288095238095239
Evaluated Accuracy: 0.9288
Testing b_a_r = 0.0145, c_a_r = 0.1022, f_a_r = 0.9654


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.9340476190476191
Evaluated Accuracy: 0.9340

✅ Optimal Values:
   - b_a_r: 0.0118
   - c_a_r: 0.1040
   - f_a_r: 0.7799
📈 Highest Accuracy Achieved: 0.9364


In [74]:
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_olchiki/resnet_resluts/base/5_w_10_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/secondary/5_w_10_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/amul/5_w_5_10_f", test_loader, M3, criterion)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_olchiki_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.8098 ± 0.0727  0.8214 ± 0.0711  0.8098 ± 0.0727


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.6790 ± 0.0583  0.6879 ± 0.0631  0.6790 ± 0.0583
---Accu-----pre----rec---------> 0.9364 ± 0.0456  0.9386 ± 0.0436  0.9364 ± 0.0456


In [50]:
ls /home/asufian/Desktop/output_olchiki/lol

5_w_10_s.txt  5_w_10_s.xlsx


In [76]:
clss_8=[
    ['Olchiki_2_80_c10', 'Olchiki_2_80_c25', 'Olchiki_2_80_c27', 'Olchiki_2_80_c13', 'Olchiki_2_80_c8', 'Olchiki_2_80_c11', 'Olchiki_2_80_c15', 'Olchiki_2_80_c5'],
    ['Olchiki_2_80_c1', 'Olchiki_2_80_c27', 'Olchiki_2_80_c8', 'Olchiki_2_80_c4', 'Olchiki_2_80_c18', 'Olchiki_2_80_c2', 'Olchiki_2_80_c29', 'Olchiki_2_80_c26'],
    ['Olchiki_2_80_c23', 'Olchiki_2_80_c11', 'Olchiki_2_80_c25', 'Olchiki_2_80_c29', 'Olchiki_2_80_c21', 'Olchiki_2_80_c27', 'Olchiki_2_80_c3', 'Olchiki_2_80_c7'],
    ['Olchiki_2_80_c23', 'Olchiki_2_80_c4', 'Olchiki_2_80_c26', 'Olchiki_2_80_c2', 'Olchiki_2_80_c17', 'Olchiki_2_80_c30', 'Olchiki_2_80_c18', 'Olchiki_2_80_c9'],
    ['Olchiki_2_80_c6', 'Olchiki_2_80_c12', 'Olchiki_2_80_c24', 'Olchiki_2_80_c18', 'Olchiki_2_80_c22', 'Olchiki_2_80_c10', 'Olchiki_2_80_c26', 'Olchiki_2_80_c16'],
     ['Olchiki_2_80_c6', 'Olchiki_2_80_c26', 'Olchiki_2_80_c8', 'Olchiki_2_80_c12', 'Olchiki_2_80_c14', 'Olchiki_2_80_c16', 'Olchiki_2_80_c25', 'Olchiki_2_80_c4'],
    ['Olchiki_2_80_c24', 'Olchiki_2_80_c26', 'Olchiki_2_80_c21', 'Olchiki_2_80_c7', 'Olchiki_2_80_c12', 'Olchiki_2_80_c25', 'Olchiki_2_80_c2', 'Olchiki_2_80_c5'],
    ['Olchiki_2_80_c5', 'Olchiki_2_80_c18', 'Olchiki_2_80_c30', 'Olchiki_2_80_c7', 'Olchiki_2_80_c9', 'Olchiki_2_80_c16', 'Olchiki_2_80_c6', 'Olchiki_2_80_c24'],
    ['Olchiki_2_80_c27', 'Olchiki_2_80_c5', 'Olchiki_2_80_c3', 'Olchiki_2_80_c12', 'Olchiki_2_80_c30', 'Olchiki_2_80_c6', 'Olchiki_2_80_c18', 'Olchiki_2_80_c10'],
    ['Olchiki_2_80_c26', 'Olchiki_2_80_c20', 'Olchiki_2_80_c1', 'Olchiki_2_80_c6', 'Olchiki_2_80_c13', 'Olchiki_2_80_c23', 'Olchiki_2_80_c30', 'Olchiki_2_80_c5'],
    ['Olchiki_2_80_c12', 'Olchiki_2_80_c8', 'Olchiki_2_80_c11', 'Olchiki_2_80_c14', 'Olchiki_2_80_c25', 'Olchiki_2_80_c4', 'Olchiki_2_80_c2', 'Olchiki_2_80_c29'],
    ['Olchiki_2_80_c17', 'Olchiki_2_80_c14', 'Olchiki_2_80_c20', 'Olchiki_2_80_c29', 'Olchiki_2_80_c8', 'Olchiki_2_80_c11', 'Olchiki_2_80_c2', 'Olchiki_2_80_c28'],
    
  

     ]

clss_support_imagesss_10_shot=[
    [[6, 16, 45, 19, 27, 44, 23, 60, 42, 13], [60, 32, 27, 28, 65, 66, 44, 3, 21, 34], [8, 74, 70, 6, 14, 23, 69, 75, 39, 21], [42, 58, 71, 17, 35, 1, 75, 40, 56, 44], [10, 51, 55, 15, 75, 48, 6, 56, 63, 19], [11, 47, 24, 10, 15, 37, 63, 48, 33, 22], [72, 58, 3, 32, 41, 51, 15, 46, 0, 52], [61, 6, 76, 62, 45, 51, 55, 72, 53, 40]],
    [[3, 23, 33, 18, 2, 25, 50, 6, 36, 68], [55, 79, 3, 0, 15, 53, 4, 46, 2, 44], [11, 9, 62, 15, 36, 37, 24, 38, 13, 49], [34, 26, 31, 70, 44, 17, 5, 12, 67, 15], [23, 72, 65, 58, 5, 42, 11, 22, 15, 69], [40, 65, 22, 56, 7, 69, 66, 57, 54, 48], [13, 40, 20, 75, 71, 45, 12, 32, 56, 79], [22, 55, 51, 69, 64, 18, 60, 31, 30, 11]],
    [[62, 36, 4, 1, 3, 32, 31, 33, 59, 7], [25, 62, 6, 45, 53, 7, 71, 72, 61, 23], [21, 68, 67, 64, 36, 13, 33, 2, 25, 24], [59, 76, 28, 11, 10, 39, 51, 7, 22, 25], [11, 29, 59, 64, 40, 63, 27, 37, 22, 9], [42, 51, 50, 26, 56, 79, 8, 7, 15, 0], [28, 43, 56, 19, 48, 20, 50, 6, 73, 10], [74, 70, 67, 65, 72, 47, 62, 26, 49, 10]],
    [[22, 58, 20, 1, 55, 40, 6, 18, 62, 33], [9, 28, 77, 31, 4, 37, 17, 29, 64, 40], [30, 7, 9, 20, 74, 79, 43, 2, 24, 55], [18, 13, 20, 19, 5, 4, 0, 39, 25, 34], [11, 28, 38, 72, 9, 47, 6, 62, 43, 64], [23, 26, 65, 54, 24, 27, 15, 48, 41, 28], [20, 55, 53, 3, 24, 8, 13, 50, 68, 35], [32, 33, 24, 51, 79, 39, 57, 37, 42, 60]],
    [[38, 30, 53, 3, 0, 68, 18, 47, 28, 32], [55, 22, 0, 37, 75, 7, 64, 12, 34, 62], [6, 25, 44, 67, 77, 8, 70, 76, 3, 53], [70, 23, 59, 30, 37, 39, 12, 34, 53, 56], [77, 25, 60, 33, 43, 52, 75, 26, 36, 61], [29, 76, 1, 56, 55, 0, 3, 19, 18, 40], [40, 41, 45, 19, 6, 27, 18, 15, 43, 56], [72, 52, 24, 42, 66, 60, 79, 10, 11, 73]],
     [[37, 66, 29, 79, 19, 72, 63, 49, 20, 4], [51, 47, 75, 16, 17, 61, 3, 78, 77, 24], [1, 27, 34, 16, 30, 48, 75, 12, 10, 33], [24, 66, 65, 71, 8, 12, 56, 58, 16, 30], [26, 23, 74, 35, 40, 8, 38, 37, 42, 66], [30, 21, 33, 0, 28, 20, 19, 79, 12, 45], [22, 5, 32, 77, 40, 67, 46, 7, 42, 56], [14, 70, 63, 6, 23, 5, 30, 72, 26, 41]],
    [[29, 43, 70, 46, 11, 58, 37, 48, 21, 23], [18, 42, 64, 14, 38, 6, 16, 21, 29, 67], [15, 72, 56, 69, 78, 39, 29, 49, 27, 36], [67, 54, 16, 26, 58, 17, 68, 42, 19, 76], [60, 75, 74, 7, 2, 47, 34, 25, 44, 63], [28, 62, 59, 68, 65, 16, 72, 48, 42, 10], [44, 55, 58, 42, 78, 24, 9, 32, 16, 73], [17, 34, 19, 32, 69, 75, 46, 45, 39, 18]],
    [[58, 54, 29, 40, 11, 43, 30, 44, 70, 3], [75, 21, 6, 36, 48, 70, 64, 30, 32, 59], [1, 70, 41, 69, 67, 19, 16, 77, 45, 53], [19, 5, 11, 16, 34, 14, 70, 71, 69, 33], [3, 14, 75, 28, 1, 5, 15, 72, 40, 60], [16, 22, 38, 77, 39, 7, 79, 9, 58, 46], [53, 36, 62, 75, 37, 45, 76, 74, 58, 70], [74, 33, 62, 8, 73, 75, 38, 0, 20, 12]],
    [[79, 67, 1, 32, 11, 7, 59, 26, 50, 57], [11, 24, 16, 74, 39, 78, 41, 6, 36, 27], [27, 41, 58, 71, 39, 42, 25, 13, 24, 73], [47, 67, 42, 35, 3, 24, 53, 43, 39, 16], [15, 14, 2, 23, 53, 18, 25, 48, 4, 66], [66, 36, 68, 14, 63, 13, 48, 15, 56, 20], [13, 72, 12, 4, 76, 29, 45, 53, 59, 70], [73, 52, 24, 26, 42, 5, 0, 63, 2, 45]],
     [[42, 35, 43, 7, 20, 30, 17, 2, 76, 45], [6, 24, 17, 30, 68, 12, 78, 7, 50, 55], [12, 30, 1, 74, 69, 16, 15, 17, 34, 9], [64, 37, 66, 77, 45, 49, 78, 75, 31, 57], [21, 49, 36, 12, 52, 10, 60, 50, 31, 77], [28, 7, 72, 49, 1, 67, 10, 0, 38, 32], [69, 59, 58, 22, 15, 14, 60, 7, 21, 35], [45, 1, 62, 46, 35, 12, 66, 17, 64, 27]],
    [[14, 68, 70, 55, 67, 58, 79, 62, 2, 31], [9, 1, 10, 26, 25, 49, 35, 6, 0, 33], [56, 62, 75, 31, 25, 71, 40, 12, 39, 6], [72, 75, 40, 34, 8, 63, 79, 20, 73, 12], [36, 70, 50, 37, 69, 66, 54, 75, 8, 55], [18, 27, 8, 65, 33, 57, 28, 56, 0, 60], [51, 41, 4, 10, 57, 21, 11, 3, 44, 79], [13, 52, 5, 71, 9, 68, 28, 46, 63, 12]],
     [[25, 79, 77, 21, 73, 38, 52, 57, 75, 31], [61, 76, 63, 53, 27, 49, 0, 38, 21, 31], [77, 53, 4, 0, 14, 29, 34, 30, 39, 3], [36, 50, 8, 30, 21, 55, 54, 71, 4, 12], [16, 4, 44, 36, 66, 30, 79, 58, 7, 24], [45, 42, 56, 59, 57, 22, 79, 69, 36, 70], [27, 77, 31, 32, 61, 55, 9, 10, 44, 50], [71, 25, 45, 11, 16, 72, 79, 47, 32, 63]]
]
clss_support_imagesss_5_shot=[
    [[6, 16, 45, 19, 27], [60, 32, 27, 28, 65], [8, 74, 70, 6, 14], [42, 58, 71, 17, 35], [10, 51, 55, 15, 75], [11, 47, 24, 10, 15], [72, 58, 3, 32, 41], [61, 6, 76, 62, 45]],
    [[3, 23, 33, 18, 2], [55, 79, 3, 0, 15], [11, 9, 62, 15, 36], [34, 26, 31, 70, 44], [23, 72, 65, 58, 5], [40, 65, 22, 56, 7], [13, 40, 20, 75, 71], [22, 55, 51, 69, 64]],
    [[62, 36, 4, 1, 3], [25, 62, 6, 45, 53], [21, 68, 67, 64, 36], [59, 76, 28, 11, 10], [11, 29, 59, 64, 40], [42, 51, 50, 26, 56], [28, 43, 56, 19, 48], [74, 70, 67, 65, 72]],
    [[22, 58, 20, 1, 55], [9, 28, 77, 31, 4], [30, 7, 9, 20, 74], [18, 13, 20, 19, 5], [11, 28, 38, 72, 9], [23, 26, 65, 54, 24], [20, 55, 53, 3, 24], [32, 33, 24, 51, 79]],
    [[38, 30, 53, 3, 0], [55, 22, 0, 37, 75], [6, 25, 44, 67, 77], [70, 23, 59, 30, 37], [77, 25, 60, 33, 43], [29, 76, 1, 56, 55], [40, 41, 45, 19, 6], [72, 52, 24, 42, 66]],
    [[37, 66, 29, 79, 19], [51, 47, 75, 16, 17], [1, 27, 34, 16, 30], [24, 66, 65, 71, 8], [26, 23, 74, 35, 40], [30, 21, 33, 0, 28], [22, 5, 32, 77, 40], [14, 70, 63, 6, 23]],
    [[29, 43, 70, 46, 11], [18, 42, 64, 14, 38], [15, 72, 56, 69, 78], [67, 54, 16, 26, 58], [60, 75, 74, 7, 2], [28, 62, 59, 68, 65], [44, 55, 58, 42, 78], [17, 34, 19, 32, 69]],
    [[58, 54, 29, 40, 11], [75, 21, 6, 36, 48], [1, 70, 41, 69, 67], [19, 5, 11, 16, 34], [3, 14, 75, 28, 1], [16, 22, 38, 77, 39], [53, 36, 62, 75, 37], [74, 33, 62, 8, 73]],
    [[79, 67, 1, 32, 11], [11, 24, 16, 74, 39], [27, 41, 58, 71, 39], [47, 67, 42, 35, 3], [15, 14, 2, 23, 53], [66, 36, 68, 14, 63], [13, 72, 12, 4, 76], [73, 52, 24, 26, 42]],
    [[42, 35, 43, 7, 20], [6, 24, 17, 30, 68], [12, 30, 1, 74, 69], [64, 37, 66, 77, 45], [21, 49, 36, 12, 52], [28, 7, 72, 49, 1], [69, 59, 58, 22, 15], [45, 1, 62, 46, 35]],
    [[14, 68, 70, 55, 67], [9, 1, 10, 26, 25], [56, 62, 75, 31, 25], [72, 75, 40, 34, 8], [36, 70, 50, 37, 69], [18, 27, 8, 65, 33], [51, 41, 4, 10, 57], [13, 52, 5, 71, 9]],
    [[25, 79, 77, 21, 73], [61, 76, 63, 53, 27], [77, 53, 4, 0, 14], [36, 50, 8, 30, 21], [16, 4, 44, 36, 66], [45, 42, 56, 59, 57], [27, 77, 31, 32, 61], [71, 25, 45, 11, 16]]
]


clss_support_imagesss_1_shot=[
    [[6], [60], [8], [42], [10], [11], [72], [61]],
    [[3], [55], [11], [34], [23], [40], [13], [22]],
    [[62], [25], [21], [59], [11], [42], [28], [74]],
     [[22], [9], [30], [18], [11], [23], [20], [32]],
    [[38], [55], [6], [70], [77], [29], [40], [72]],
    [[37], [51], [1], [24], [26], [30], [22], [14]],
    [[29], [18], [15], [67], [60], [28], [44], [17]],
    [[58], [75], [1], [19], [3], [16], [53], [74]],
    [[79], [11], [27], [47], [15], [66], [13], [73]],
    [[42], [6], [12], [64], [21], [28], [69], [45]],
    [[14], [9], [56], [72], [36], [18], [51], [13]],
    [[25], [61], [77], [36], [16], [45], [27], [71]]
 






]



In [77]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelr_res18.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/olchiki/ressecondary/model_1/model_B_olchiki_1-shot_res.pth',map_location=torch.device('cpu')))
torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [78]:


def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

           
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

            
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [79]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.636867088607595
Evaluated Accuracy: 0.6369
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000
----------------------> 0.5644778481012659
Evaluated Accuracy: 0.5645
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429
----------------------> 0.6181434599156118
Evaluated Accuracy: 0.6181
Testing b_a_r = 0.6509, c_a_r = 0.0564, f_a_r = 0.7220
----------------------> 0.6971255274261603
Evaluated Accuracy: 0.6971
Testing b_a_r = 0.9386, c_a_r = 0.0008, f_a_r = 0.9922
----------------------> 0.6824894514767932
Evaluated Accuracy: 0.6825
Testing b_a_r = 0.0000, c_a_r = 0.0000, f_a_r = 1.0000


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.6760284810126581
Evaluated Accuracy: 0.6760
Testing b_a_r = 0.2213, c_a_r = 1.0000, f_a_r = 0.9428


...

----------------------> 0.4030854430379747
Evaluated Accuracy: 0.4031
Testing b_a_r = 0.0037, c_a_r = 0.0254, f_a_r = 0.8168


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.70042194092827
Evaluated Accuracy: 0.7004
Testing b_a_r = 0.9941, c_a_r = 0.1123, f_a_r = 0.6539


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.7213871308016877
Evaluated Accuracy: 0.7214

✅ Optimal Values:
   - b_a_r: 0.9567
   - c_a_r: 0.1018
   - f_a_r: 0.7030
📈 Highest Accuracy Achieved: 0.7231


In [80]:
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_olchiki/resnet_resluts/base/8_w_1_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/secondary/8_w_1_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/amul/8_w_1_s_f", test_loader, M3, criterion)


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5993 ± 0.0565  0.6236 ± 0.0574  0.5993 ± 0.0565


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.4285 ± 0.0644  0.4663 ± 0.0691  0.4285 ± 0.0644
---Accu-----pre----rec---------> 0.7231 ± 0.0661  0.7440 ± 0.0758  0.7231 ± 0.0661


In [81]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelr_res18.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/olchiki/ressecondary/model_5/model_B_olchiki_5-shot_res.pth',map_location=torch.device('cpu')))

torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [82]:
def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

           
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

            
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [83]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.8236111111111111
Evaluated Accuracy: 0.8236
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000
----------------------> 0.6056944444444444
Evaluated Accuracy: 0.6057
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429
----------------------> 0.7254166666666667
Evaluated Accuracy: 0.7254
Testing b_a_r = 0.6509, c_a_r = 0.0564, f_a_r = 0.7220
----------------------> 0.7895833333333333
Evaluated Accuracy: 0.7896
Testing b_a_r = 0.9386, c_a_r = 0.0008, f_a_r = 0.9922
----------------------> 0.7531944444444445
Evaluated Accuracy: 0.7532
Testing b_a_r = 0.0250, c_a_r = 1.0000, f_a_r = 0.4630


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.3630555555555555
Evaluated Accuracy: 0.3631
Testing b_a_r = 0.0000, c_a_r = 0.1539, f_a_r = 0.0000


...

----------------------> 0.7975
Evaluated Accuracy: 0.7975
Testing b_a_r = 0.8591, c_a_r = 0.6123, f_a_r = 0.9993


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.4266666666666667
Evaluated Accuracy: 0.4267
Testing b_a_r = 0.0007, c_a_r = 0.1724, f_a_r = 0.8439


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.8361111111111109
Evaluated Accuracy: 0.8361

✅ Optimal Values:
   - b_a_r: 0.0067
   - c_a_r: 0.1555
   - f_a_r: 0.7886
📈 Highest Accuracy Achieved: 0.8418


In [84]:
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_olchiki/resnet_resluts/base/8_w_5_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/secondary/8_w_5_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/amul/8_w_5_s_f", test_loader, M3, criterion)


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.7114 ± 0.0670  0.7267 ± 0.0709  0.7114 ± 0.0670


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5714 ± 0.0683  0.5837 ± 0.0774  0.5714 ± 0.0683
---Accu-----pre----rec---------> 0.8418 ± 0.0481  0.8504 ± 0.0503  0.8418 ± 0.0481


In [85]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelr_res18.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/olchiki/ressecondary/model_10/model_B_olchiki_10-shot_res.pth',map_location=torch.device('cpu')))

torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [86]:
def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

           
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

            
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [87]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=80
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.8497023809523809
Evaluated Accuracy: 0.8497
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000
----------------------> 0.5840773809523809
Evaluated Accuracy: 0.5841
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429
----------------------> 0.747470238095238
Evaluated Accuracy: 0.7475
Testing b_a_r = 0.6509, c_a_r = 0.0564, f_a_r = 0.7220
----------------------> 0.8492559523809523
Evaluated Accuracy: 0.8493
Testing b_a_r = 0.9386, c_a_r = 0.0008, f_a_r = 0.9922
----------------------> 0.7992559523809524
Evaluated Accuracy: 0.7993
Testing b_a_r = 0.0250, c_a_r = 1.0000, f_a_r = 0.4630


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.2921130952380952
Evaluated Accuracy: 0.2921
Testing b_a_r = 0.0000, c_a_r = 0.1302, f_a_r = 0.0000


...

----------------------> 0.8101190476190475
Evaluated Accuracy: 0.8101
Testing b_a_r = 0.9927, c_a_r = 0.2202, f_a_r = 0.5487


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.8642857142857144
Evaluated Accuracy: 0.8643
Testing b_a_r = 0.0380, c_a_r = 0.2683, f_a_r = 0.9678


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.7895833333333333
Evaluated Accuracy: 0.7896

✅ Optimal Values:
   - b_a_r: 0.0000
   - c_a_r: 0.0828
   - f_a_r: 1.0000
📈 Highest Accuracy Achieved: 0.8720


In [88]:
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_olchiki/resnet_resluts/base/8_w_10_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/secondary/8_w_10_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/amul/8_w_10_s_f", test_loader, M3, criterion)


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.7259 ± 0.0547  0.7357 ± 0.0563  0.7259 ± 0.0547


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5624 ± 0.0649  0.5727 ± 0.0628  0.5624 ± 0.0649
---Accu-----pre----rec---------> 0.8720 ± 0.0406  0.8778 ± 0.0412  0.8720 ± 0.0406


In [ ]:
###############################################   12 way 1 shot

In [89]:
clss_support_imagesss_10_shot=[
[[5, 47, 72, 9, 32, 44, 13, 55, 77, 0], [13, 10, 72, 9, 71, 23, 18, 20, 46, 64], [10, 69, 21, 5, 38, 11, 7, 64, 59, 4], [16, 18, 21, 29, 43, 9, 19, 66, 63, 57], [69, 40, 4, 60, 23, 0, 30, 74, 17, 35], [48, 35, 36, 64, 0, 26, 24, 71, 8, 76], [0, 52, 22, 77, 9, 59, 29, 21, 75, 53], [17, 8, 69, 11, 44, 73, 77, 0, 59, 46], [31, 41, 17, 37, 16, 54, 40, 39, 25, 42], [19, 65, 62, 13, 5, 67, 4, 69, 76, 0], [44, 74, 53, 28, 58, 15, 43, 55, 34, 50], [29, 63, 21, 27, 39, 67, 33, 71, 1, 19]],
    [[79, 47, 13, 50, 68, 9, 4, 12, 62, 24], [16, 54, 8, 23, 12, 37, 31, 72, 48, 52], [63, 70, 20, 58, 12, 26, 27, 59, 76, 52], [42, 15, 55, 59, 22, 76, 79, 6, 11, 58], [60, 62, 15, 54, 72, 1, 38, 77, 66, 22], [63, 36, 72, 56, 10, 24, 55, 4, 6, 57], [8, 64, 61, 26, 77, 56, 43, 20, 41, 54], [66, 25, 52, 23, 60, 33, 31, 12, 67, 5], [62, 13, 77, 21, 65, 59, 41, 45, 58, 71], [20, 10, 69, 63, 11, 9, 68, 57, 74, 75], [40, 8, 66, 31, 13, 54, 63, 4, 64, 29], [7, 14, 42, 66, 73, 41, 75, 44, 5, 70]],
    [[51, 46, 56, 19, 16, 36, 25, 29, 53, 12], [6, 11, 68, 48, 35, 46, 78, 23, 39, 15], [34, 2, 32, 21, 18, 52, 74, 71, 12, 24], [64, 13, 29, 43, 52, 11, 37, 50, 49, 17], [24, 74, 20, 25, 61, 45, 54, 4, 78, 59], [53, 50, 12, 39, 66, 9, 73, 15, 11, 44], [38, 76, 65, 28, 24, 58, 36, 56, 18, 68], [43, 72, 4, 8, 73, 63, 19, 51, 60, 32], [19, 44, 65, 7, 2, 71, 66, 35, 50, 41], [44, 35, 14, 34, 29, 45, 23, 26, 41, 43], [35, 59, 1, 46, 37, 41, 12, 38, 44, 56], [28, 37, 47, 19, 59, 4, 52, 63, 0, 7]],
    [[79, 60, 39, 74, 70, 48, 8, 11, 66, 33], [6, 74, 64, 9, 27, 70, 47, 19, 24, 63], [48, 62, 69, 51, 55, 41, 6, 59, 50, 18], [4, 54, 72, 66, 20, 69, 62, 58, 6, 53], [28, 20, 46, 52, 72, 11, 73, 9, 22, 57], [13, 33, 62, 59, 72, 37, 76, 5, 16, 22], [67, 11, 70, 58, 50, 6, 5, 17, 38, 28], [16, 27, 41, 2, 0, 17, 8, 1, 32, 71], [70, 52, 0, 67, 74, 23, 62, 51, 55, 63], [27, 46, 79, 43, 44, 67, 55, 72, 58, 54], [9, 48, 37, 68, 56, 50, 61, 27, 47, 77], [15, 37, 63, 69, 58, 70, 19, 62, 17, 78]],
    [[9, 35, 16, 21, 28, 47, 17, 6, 53, 68], [7, 53, 31, 51, 30, 72, 42, 9, 62, 34], [29, 59, 33, 75, 45, 0, 28, 76, 51, 66], [50, 67, 39, 68, 1, 47, 46, 58, 11, 34], [71, 14, 40, 15, 32, 70, 73, 57, 6, 67], [7, 63, 20, 6, 8, 5, 42, 68, 50, 59], [6, 65, 2, 43, 69, 59, 37, 16, 1, 20], [3, 12, 37, 33, 17, 4, 24, 18, 46, 41], [26, 9, 35, 39, 74, 8, 16, 22, 64, 53], [70, 56, 67, 6, 10, 4, 19, 57, 11, 31], [19, 32, 57, 39, 54, 55, 18, 74, 51, 35], [64, 10, 58, 18, 23, 8, 31, 37, 20, 9]],
    [[4, 50, 41, 48, 72, 7, 79, 43, 65, 69], [38, 51, 64, 35, 43, 0, 5, 58, 67, 34], [65, 16, 1, 6, 52, 34, 3, 44, 38, 31], [32, 12, 74, 14, 29, 13, 31, 1, 27, 76], [78, 54, 53, 33, 67, 71, 22, 31, 25, 45], [65, 64, 43, 49, 74, 8, 45, 48, 39, 69], [12, 73, 32, 10, 63, 79, 11, 69, 54, 35], [33, 48, 70, 71, 14, 26, 28, 17, 3, 67], [17, 22, 29, 77, 14, 75, 52, 31, 50, 26], [19, 41, 67, 4, 53, 77, 17, 43, 38, 16], [75, 38, 24, 18, 23, 39, 5, 4, 72, 50], [27, 43, 64, 47, 50, 38, 14, 2, 33, 37]],
    [[47, 13, 68, 62, 58, 39, 24, 26, 45, 74], [3, 2, 76, 47, 7, 17, 24, 10, 38, 59], [5, 10, 20, 4, 1, 38, 24, 64, 3, 57], [39, 18, 64, 20, 78, 56, 76, 28, 36, 24], [35, 38, 48, 18, 8, 25, 75, 69, 46, 79], [45, 36, 60, 77, 4, 27, 10, 29, 68, 37], [35, 58, 31, 79, 28, 74, 71, 15, 69, 11], [29, 21, 70, 9, 26, 51, 6, 31, 76, 65], [33, 4, 20, 49, 9, 2, 32, 26, 59, 64], [25, 19, 52, 22, 0, 4, 10, 2, 73, 71], [5, 9, 54, 1, 14, 45, 25, 7, 37, 17], [0, 39, 53, 31, 78, 37, 27, 9, 71, 23]],
    [[31, 77, 36, 11, 43, 17, 7, 34, 9, 32], [58, 38, 24, 68, 5, 76, 63, 65, 42, 48], [31, 12, 65, 26, 15, 56, 30, 69, 7, 53], [46, 71, 35, 37, 33, 75, 4, 9, 76, 5], [66, 60, 53, 32, 31, 78, 10, 48, 39, 62], [7, 25, 51, 27, 19, 12, 17, 30, 79, 33], [69, 39, 9, 17, 8, 41, 70, 5, 61, 4], [26, 58, 54, 51, 23, 49, 13, 34, 72, 10], [49, 76, 39, 26, 46, 70, 69, 3, 21, 12], [15, 20, 24, 53, 65, 54, 0, 69, 22, 40], [0, 7, 34, 69, 67, 57, 45, 76, 25, 30], [9, 8, 16, 12, 11, 22, 58, 26, 34, 37]],
    [[5, 34, 32, 26, 35, 36, 31, 21, 56, 8], [61, 56, 25, 29, 64, 45, 74, 24, 18, 0], [22, 46, 10, 17, 66, 50, 21, 42, 40, 73], [58, 2, 67, 78, 26, 48, 57, 41, 45, 59], [48, 15, 3, 23, 26, 37, 35, 49, 12, 39], [20, 68, 56, 30, 33, 77, 16, 17, 10, 35], [0, 16, 1, 77, 48, 68, 67, 14, 8, 22], [74, 63, 64, 21, 10, 16, 66, 54, 28, 23], [8, 40, 35, 60, 71, 34, 54, 26, 64, 37], [70, 23, 58, 25, 65, 30, 12, 31, 4, 61], [33, 51, 74, 22, 9, 53, 72, 41, 69, 60], [76, 1, 28, 68, 49, 14, 4, 12, 48, 36]],
    [[46, 29, 28, 79, 53, 70, 38, 15, 37, 17], [38, 43, 31, 51, 28, 54, 62, 59, 21, 49], [19, 51, 49, 30, 45, 15, 64, 46, 56, 75], [79, 9, 53, 35, 73, 44, 22, 75, 68, 63], [69, 46, 53, 58, 21, 2, 44, 43, 11, 41], [70, 71, 44, 45, 16, 18, 5, 53, 7, 38], [78, 30, 0, 43, 56, 22, 11, 29, 21, 23], [62, 76, 5, 47, 3, 40, 2, 68, 34, 65], [68, 56, 40, 64, 42, 8, 52, 73, 63, 2], [24, 4, 28, 66, 60, 48, 70, 72, 79, 23], [67, 24, 70, 77, 16, 43, 14, 64, 26, 17], [79, 6, 18, 4, 63, 55, 77, 58, 47, 54]],
    [[74, 79, 0, 5, 19, 73, 37, 32, 42, 58], [15, 44, 17, 3, 14, 36, 24, 6, 56, 77], [13, 22, 7, 69, 10, 41, 56, 25, 15, 48], [17, 57, 25, 60, 43, 24, 53, 9, 10, 54], [71, 38, 25, 4, 64, 78, 39, 53, 41, 69], [57, 63, 6, 32, 34, 25, 20, 23, 39, 76], [69, 48, 43, 7, 19, 68, 12, 77, 36, 65], [44, 55, 74, 46, 51, 21, 38, 7, 11, 61], [54, 12, 78, 8, 44, 49, 46, 35, 45, 72], [69, 8, 35, 47, 49, 62, 42, 32, 4, 70], [76, 11, 37, 60, 57, 18, 30, 7, 5, 39], [67, 37, 63, 68, 20, 24, 51, 62, 28, 2]],
    [[66, 55, 74, 37, 40, 1, 21, 46, 34, 28], [31, 59, 66, 69, 16, 6, 37, 11, 60, 58], [58, 48, 35, 65, 67, 2, 60, 45, 17, 6], [49, 63, 9, 74, 26, 79, 11, 46, 56, 59], [22, 35, 66, 42, 10, 75, 28, 1, 39, 67], [23, 50, 41, 17, 71, 32, 53, 27, 56, 21], [29, 67, 37, 69, 50, 8, 40, 17, 59, 32], [0, 79, 20, 68, 63, 66, 50, 26, 57, 44], [38, 46, 70, 61, 4, 43, 39, 3, 50, 55], [65, 58, 51, 59, 71, 74, 39, 34, 60, 23], [30, 26, 72, 25, 27, 23, 42, 24, 53, 1], [31, 22, 34, 7, 0, 5, 42, 36, 49, 45]]
    
  ]


clss_support_imagesss_5_shot=[
    [[5, 47, 72, 9, 32], [13, 10, 72, 9, 71], [10, 69, 21, 5, 38], [16, 18, 21, 29, 43], [69, 40, 4, 60, 23], [48, 35, 36, 64, 0], [0, 52, 22, 77, 9], [17, 8, 69, 11, 44], [31, 41, 17, 37, 16], [19, 65, 62, 13, 5], [44, 74, 53, 28, 58], [29, 63, 21, 27, 39]],
    [[79, 47, 13, 50, 68], [16, 54, 8, 23, 12], [63, 70, 20, 58, 12], [42, 15, 55, 59, 22], [60, 62, 15, 54, 72], [63, 36, 72, 56, 10], [8, 64, 61, 26, 77], [66, 25, 52, 23, 60], [62, 13, 77, 21, 65], [20, 10, 69, 63, 11], [40, 8, 66, 31, 13], [7, 14, 42, 66, 73]],
    [[51, 46, 56, 19, 16], [6, 11, 68, 48, 35], [34, 2, 32, 21, 18], [64, 13, 29, 43, 52], [24, 74, 20, 25, 61], [53, 50, 12, 39, 66], [38, 76, 65, 28, 24], [43, 72, 4, 8, 73], [19, 44, 65, 7, 2], [44, 35, 14, 34, 29], [35, 59, 1, 46, 37], [28, 37, 47, 19, 59]],
    [[79, 60, 39, 74, 70], [6, 74, 64, 9, 27], [48, 62, 69, 51, 55], [4, 54, 72, 66, 20], [28, 20, 46, 52, 72], [13, 33, 62, 59, 72], [67, 11, 70, 58, 50], [16, 27, 41, 2, 0], [70, 52, 0, 67, 74], [27, 46, 79, 43, 44], [9, 48, 37, 68, 56], [15, 37, 63, 69, 58]],
    [[9, 35, 16, 21, 28], [7, 53, 31, 51, 30], [29, 59, 33, 75, 45], [50, 67, 39, 68, 1], [71, 14, 40, 15, 32], [7, 63, 20, 6, 8], [6, 65, 2, 43, 69], [3, 12, 37, 33, 17], [26, 9, 35, 39, 74], [70, 56, 67, 6, 10], [19, 32, 57, 39, 54], [64, 10, 58, 18, 23]],
    [[4, 50, 41, 48, 72], [38, 51, 64, 35, 43], [65, 16, 1, 6, 52], [32, 12, 74, 14, 29], [78, 54, 53, 33, 67], [65, 64, 43, 49, 74], [12, 73, 32, 10, 63], [33, 48, 70, 71, 14], [17, 22, 29, 77, 14], [19, 41, 67, 4, 53], [75, 38, 24, 18, 23], [27, 43, 64, 47, 50]],
    [[47, 13, 68, 62, 58], [3, 2, 76, 47, 7], [5, 10, 20, 4, 1], [39, 18, 64, 20, 78], [35, 38, 48, 18, 8], [45, 36, 60, 77, 4], [35, 58, 31, 79, 28], [29, 21, 70, 9, 26], [33, 4, 20, 49, 9], [25, 19, 52, 22, 0], [5, 9, 54, 1, 14], [0, 39, 53, 31, 78]],
    [[31, 77, 36, 11, 43], [58, 38, 24, 68, 5], [31, 12, 65, 26, 15], [46, 71, 35, 37, 33], [66, 60, 53, 32, 31], [7, 25, 51, 27, 19], [69, 39, 9, 17, 8], [26, 58, 54, 51, 23], [49, 76, 39, 26, 46], [15, 20, 24, 53, 65], [0, 7, 34, 69, 67], [9, 8, 16, 12, 11]],
    [[5, 34, 32, 26, 35], [61, 56, 25, 29, 64], [22, 46, 10, 17, 66], [58, 2, 67, 78, 26], [48, 15, 3, 23, 26], [20, 68, 56, 30, 33], [0, 16, 1, 77, 48], [74, 63, 64, 21, 10], [8, 40, 35, 60, 71], [70, 23, 58, 25, 65], [33, 51, 74, 22, 9], [76, 1, 28, 68, 49]],
    [[46, 29, 28, 79, 53], [38, 43, 31, 51, 28], [19, 51, 49, 30, 45], [79, 9, 53, 35, 73], [69, 46, 53, 58, 21], [70, 71, 44, 45, 16], [78, 30, 0, 43, 56], [62, 76, 5, 47, 3], [68, 56, 40, 64, 42], [24, 4, 28, 66, 60], [67, 24, 70, 77, 16], [79, 6, 18, 4, 63]],
    [[74, 79, 0, 5, 19], [15, 44, 17, 3, 14], [13, 22, 7, 69, 10], [17, 57, 25, 60, 43], [71, 38, 25, 4, 64], [57, 63, 6, 32, 34], [69, 48, 43, 7, 19], [44, 55, 74, 46, 51], [54, 12, 78, 8, 44], [69, 8, 35, 47, 49], [76, 11, 37, 60, 57], [67, 37, 63, 68, 20]],
    [[66, 55, 74, 37, 40], [31, 59, 66, 69, 16], [58, 48, 35, 65, 67], [49, 63, 9, 74, 26], [22, 35, 66, 42, 10], [23, 50, 41, 17, 71], [29, 67, 37, 69, 50], [0, 79, 20, 68, 63], [38, 46, 70, 61, 4], [65, 58, 51, 59, 71], [30, 26, 72, 25, 27], [31, 22, 34, 7, 0]],
    
   ]

clss_12=[
    ['Olchiki_2_80_c23', 'Olchiki_2_80_c17', 'Olchiki_2_80_c15', 'Olchiki_2_80_c18', 'Olchiki_2_80_c12', 'Olchiki_2_80_c19', 'Olchiki_2_80_c16', 'Olchiki_2_80_c6', 'Olchiki_2_80_c27', 'Olchiki_2_80_c25', 'Olchiki_2_80_c7', 'Olchiki_2_80_c22'],
    ['Olchiki_2_80_c14', 'Olchiki_2_80_c24', 'Olchiki_2_80_c20', 'Olchiki_2_80_c11', 'Olchiki_2_80_c4', 'Olchiki_2_80_c17', 'Olchiki_2_80_c28', 'Olchiki_2_80_c8', 'Olchiki_2_80_c6', 'Olchiki_2_80_c13', 'Olchiki_2_80_c5', 'Olchiki_2_80_c12'],
    ['Olchiki_2_80_c1', 'Olchiki_2_80_c5', 'Olchiki_2_80_c8', 'Olchiki_2_80_c12', 'Olchiki_2_80_c17', 'Olchiki_2_80_c18', 'Olchiki_2_80_c13', 'Olchiki_2_80_c22', 'Olchiki_2_80_c24', 'Olchiki_2_80_c28', 'Olchiki_2_80_c27', 'Olchiki_2_80_c15'],
    ['Olchiki_2_80_c7', 'Olchiki_2_80_c12', 'Olchiki_2_80_c18', 'Olchiki_2_80_c8', 'Olchiki_2_80_c23', 'Olchiki_2_80_c4', 'Olchiki_2_80_c30', 'Olchiki_2_80_c3', 'Olchiki_2_80_c24', 'Olchiki_2_80_c28', 'Olchiki_2_80_c22', 'Olchiki_2_80_c15'],
    ['Olchiki_2_80_c8', 'Olchiki_2_80_c11', 'Olchiki_2_80_c9', 'Olchiki_2_80_c6', 'Olchiki_2_80_c21', 'Olchiki_2_80_c16', 'Olchiki_2_80_c30', 'Olchiki_2_80_c4', 'Olchiki_2_80_c5', 'Olchiki_2_80_c12', 'Olchiki_2_80_c15', 'Olchiki_2_80_c22'],
    ['Olchiki_2_80_c5', 'Olchiki_2_80_c10', 'Olchiki_2_80_c21', 'Olchiki_2_80_c18', 'Olchiki_2_80_c23', 'Olchiki_2_80_c24', 'Olchiki_2_80_c9', 'Olchiki_2_80_c14', 'Olchiki_2_80_c26', 'Olchiki_2_80_c7', 'Olchiki_2_80_c13', 'Olchiki_2_80_c2'],
    ['Olchiki_2_80_c22', 'Olchiki_2_80_c18', 'Olchiki_2_80_c5', 'Olchiki_2_80_c25', 'Olchiki_2_80_c26', 'Olchiki_2_80_c13', 'Olchiki_2_80_c7', 'Olchiki_2_80_c6', 'Olchiki_2_80_c11', 'Olchiki_2_80_c10', 'Olchiki_2_80_c4', 'Olchiki_2_80_c8'],
    ['Olchiki_2_80_c26', 'Olchiki_2_80_c15', 'Olchiki_2_80_c5', 'Olchiki_2_80_c20', 'Olchiki_2_80_c7', 'Olchiki_2_80_c9', 'Olchiki_2_80_c18', 'Olchiki_2_80_c23', 'Olchiki_2_80_c12', 'Olchiki_2_80_c25', 'Olchiki_2_80_c6', 'Olchiki_2_80_c24'],
    ['Olchiki_2_80_c3', 'Olchiki_2_80_c14', 'Olchiki_2_80_c11', 'Olchiki_2_80_c22', 'Olchiki_2_80_c26', 'Olchiki_2_80_c24', 'Olchiki_2_80_c12', 'Olchiki_2_80_c9', 'Olchiki_2_80_c28', 'Olchiki_2_80_c4', 'Olchiki_2_80_c25', 'Olchiki_2_80_c17'],
     ['Olchiki_2_80_c2', 'Olchiki_2_80_c10', 'Olchiki_2_80_c7', 'Olchiki_2_80_c29', 'Olchiki_2_80_c24', 'Olchiki_2_80_c18', 'Olchiki_2_80_c19', 'Olchiki_2_80_c8', 'Olchiki_2_80_c28', 'Olchiki_2_80_c6', 'Olchiki_2_80_c5', 'Olchiki_2_80_c12'],
    ['Olchiki_2_80_c21', 'Olchiki_2_80_c4', 'Olchiki_2_80_c15', 'Olchiki_2_80_c12', 'Olchiki_2_80_c26', 'Olchiki_2_80_c17', 'Olchiki_2_80_c8', 'Olchiki_2_80_c7', 'Olchiki_2_80_c10', 'Olchiki_2_80_c14', 'Olchiki_2_80_c19', 'Olchiki_2_80_c30'],
     ['Olchiki_2_80_c14', 'Olchiki_2_80_c8', 'Olchiki_2_80_c15', 'Olchiki_2_80_c16', 'Olchiki_2_80_c1', 'Olchiki_2_80_c6', 'Olchiki_2_80_c28', 'Olchiki_2_80_c12', 'Olchiki_2_80_c17', 'Olchiki_2_80_c23', 'Olchiki_2_80_c5', 'Olchiki_2_80_c27'],
    
     ]
clss_support_imagesss_1_shot=[
    [[5], [13], [10], [16], [69], [48], [0], [17], [31], [19], [44], [29]],
    [[79], [16], [63], [42], [60], [63], [8], [66], [62], [20], [40], [7]],
    [[51], [6], [34], [64], [24], [53], [38], [43], [19], [44], [35], [28]],
    [[79], [6], [48], [4], [28], [13], [67], [16], [70], [27], [9], [15]],
         [[9], [7], [29], [50], [71], [7], [6], [3], [26], [70], [19], [64]],
    [[4], [38], [65], [32], [78], [65], [12], [33], [17], [19], [75], [27]],
 [[47], [3], [5], [39], [35], [45], [35], [29], [33], [25], [5], [0]],
    [[31], [58], [31], [46], [66], [7], [69], [26], [49], [15], [0], [9]],
    [[5], [61], [22], [58], [48], [20], [0], [74], [8], [70], [33], [76]],
    [[46], [38], [19], [79], [69], [70], [78], [62], [68], [24], [67], [79]],
    [[74], [15], [13], [17], [71], [57], [69], [44], [54], [69], [76], [67]],
    [[66], [31], [58], [49], [22], [23], [29], [0], [38], [65], [30], [31]]
    
]















In [90]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelr_res18.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/olchiki/ressecondary/model_1/model_B_olchiki_1-shot_res.pth',map_location=torch.device('cpu')))

torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [91]:

def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

           
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

            
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [92]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.5992440225035162
Evaluated Accuracy: 0.5992
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000
----------------------> 0.5094936708860759
Evaluated Accuracy: 0.5095
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429
----------------------> 0.5636427566807314
Evaluated Accuracy: 0.5636
Testing b_a_r = 0.6509, c_a_r = 0.0564, f_a_r = 0.7220
----------------------> 0.67457805907173
Evaluated Accuracy: 0.6746
Testing b_a_r = 0.9386, c_a_r = 0.0008, f_a_r = 0.9922
----------------------> 0.6393284106891703
Evaluated Accuracy: 0.6393
Testing b_a_r = 0.1324, c_a_r = 0.0000, f_a_r = 0.9996


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.6295710267229254
Evaluated Accuracy: 0.6296
Testing b_a_r = 1.0000, c_a_r = 0.0355, f_a_r = 0.3122


...

----------------------> 0.6953234880450071
Evaluated Accuracy: 0.6953
Testing b_a_r = 0.0270, c_a_r = 0.1925, f_a_r = 0.3023


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.6308016877637131
Evaluated Accuracy: 0.6308
Testing b_a_r = 0.9757, c_a_r = 0.1277, f_a_r = 0.2777


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.6567334739803093
Evaluated Accuracy: 0.6567

✅ Optimal Values:
   - b_a_r: 0.0000
   - c_a_r: 0.1123
   - f_a_r: 0.5552
📈 Highest Accuracy Achieved: 0.6996


In [93]:
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_olchiki/resnet_resluts/base/12_w_1_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/secondary/12_w_1_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/amul/12_w_1_s_f", test_loader, M3, criterion)


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5839 ± 0.0553  0.6184 ± 0.0668  0.5839 ± 0.0553


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.3717 ± 0.0468  0.3943 ± 0.0665  0.3717 ± 0.0468
---Accu-----pre----rec---------> 0.6996 ± 0.0520  0.7226 ± 0.0524  0.6996 ± 0.0520


In [94]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelr_res18.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/olchiki/ressecondary/model_5/model_B_olchiki_5-shot_res.pth',map_location=torch.device('cpu')))


torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [95]:

def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

           
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

            
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [96]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.7956481481481482
Evaluated Accuracy: 0.7956
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000
----------------------> 0.5434259259259259
Evaluated Accuracy: 0.5434
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429
----------------------> 0.6575925925925926
Evaluated Accuracy: 0.6576
Testing b_a_r = 0.6509, c_a_r = 0.0564, f_a_r = 0.7220
----------------------> 0.7605555555555555
Evaluated Accuracy: 0.7606
Testing b_a_r = 0.9386, c_a_r = 0.0008, f_a_r = 0.9922
----------------------> 0.7197222222222223
Evaluated Accuracy: 0.7197
Testing b_a_r = 0.0250, c_a_r = 1.0000, f_a_r = 0.4630


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.28259259259259256
Evaluated Accuracy: 0.2826
Testing b_a_r = 0.0000, c_a_r = 0.1462, f_a_r = 0.0000


...

----------------------> 0.797222222222222
Evaluated Accuracy: 0.7972
Testing b_a_r = 0.0540, c_a_r = 0.8819, f_a_r = 0.0116


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.3552777777777778
Evaluated Accuracy: 0.3553
Testing b_a_r = 0.0199, c_a_r = 0.0477, f_a_r = 0.0729


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.7148148148148148
Evaluated Accuracy: 0.7148

✅ Optimal Values:
   - b_a_r: 0.0073
   - c_a_r: 0.1441
   - f_a_r: 0.6410
📈 Highest Accuracy Achieved: 0.8219


In [97]:
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_olchiki/resnet_resluts/base/12_w_5_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/secondary/12_w_5_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/amul/12_w_5_s_f", test_loader, M3, criterion)


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.6856 ± 0.0393  0.6960 ± 0.0462  0.6856 ± 0.0393


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5194 ± 0.0734  0.5270 ± 0.0659  0.5194 ± 0.0734
---Accu-----pre----rec---------> 0.8219 ± 0.0415  0.8282 ± 0.0432  0.8219 ± 0.0415


In [98]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelr_res18.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/olchiki/ressecondary/model_10/model_B_olchiki_10-shot_res.pth',map_location=torch.device('cpu')))


torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [99]:

def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

           
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

            
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [100]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.8166666666666668
Evaluated Accuracy: 0.8167
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000
----------------------> 0.5342261904761904
Evaluated Accuracy: 0.5342
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429
----------------------> 0.7042658730158732
Evaluated Accuracy: 0.7043
Testing b_a_r = 0.6509, c_a_r = 0.0564, f_a_r = 0.7220
----------------------> 0.8261904761904763
Evaluated Accuracy: 0.8262
Testing b_a_r = 0.9386, c_a_r = 0.0008, f_a_r = 0.9922
----------------------> 0.7756944444444444
Evaluated Accuracy: 0.7757
Testing b_a_r = 0.0250, c_a_r = 1.0000, f_a_r = 0.4630


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.2621031746031746
Evaluated Accuracy: 0.2621
Testing b_a_r = 0.0000, c_a_r = 0.1188, f_a_r = 0.0000


...

----------------------> 0.2876984126984127
Evaluated Accuracy: 0.2877
Testing b_a_r = 0.8966, c_a_r = 0.1168, f_a_r = 0.2691


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.7933531746031747
Evaluated Accuracy: 0.7934
Testing b_a_r = 0.0414, c_a_r = 0.1929, f_a_r = 0.6115


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------> 0.825595238095238
Evaluated Accuracy: 0.8256

✅ Optimal Values:
   - b_a_r: 0.0042
   - c_a_r: 0.0874
   - f_a_r: 0.7966
📈 Highest Accuracy Achieved: 0.8379


In [101]:
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_olchiki/resnet_resluts/base/12_w_10_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/secondary/12_w_10_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_olchiki/resnet_resluts/amul/12_w_10_s_f", test_loader, M3, criterion)


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.7173 ± 0.0372  0.7267 ± 0.0438  0.7173 ± 0.0372


/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5036 ± 0.0423  0.5161 ± 0.0440  0.5036 ± 0.0423
---Accu-----pre----rec---------> 0.8379 ± 0.0539  0.8417 ± 0.0547  0.8379 ± 0.0539
